In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import ast
from sqlalchemy import create_engine
%matplotlib inline

In [27]:
# data is stored in a sqlite database locally
engine = create_engine('sqlite:///hs_sims3.db')

df3 = pd.read_sql('SELECT * FROM EVENTS', con = engine)

In [24]:
# removing unnecessary index column
del df['index']

KeyError: 'index'

In [23]:
df.dtypes

event_key      object
event_value    object
game_id        object
player         object
timestamp      object
dtype: object

In [28]:
import cPickle
f = open('hearthstone_raw3.p','wb')
cPickle.dump(df3, f)
f.close()

In [5]:
df.head()

event_key                                        event_value  game_id  \
0      deck  ['FP1_027', 'EX1_509', 'NEW1_029', 'OG_134', '...        1   
1      deck  ['GVG_081', 'AT_106', 'OG_090', 'AT_070', 'EX1...        1   
2      deck  ['FP1_015', 'OG_133', 'EX1_572', 'EX1_558', 'N...        1   
3      deck  ['CS2_226', 'NEW1_019', 'CS2_186', 'BRM_033', ...        1   
4      deck  ['EX1_556', 'EX1_170', 'EX1_011', 'OG_254', 'E...        1   

    player                   timestamp  
0  player1  2016-06-08 18:47:53:334438  
1  player1  2016-06-08 18:47:53:370128  
2  player1  2016-06-08 18:47:53:374504  
3  player1  2016-06-08 18:47:53:381442  
4  player1  2016-06-08 18:47:53:411713

In [6]:
# converting unicode values to strings
df = df.astype(str)

# cleaning the event_values column
df.event_value.replace(to_replace = ['<', '>', '\(', '\)', 'Minion ', 'Spell ', 'Weapon ', 'Secret '],
           value = '', regex = True, inplace = True)


In [7]:
# converting strings to lists
for i in range(len(df.event_key)):
    if df.event_key[i] in ['deck', 'cards_mulliganed', 'cards_kept']:
        df.event_value[i] = ast.literal_eval(df.event_value[i])

In [8]:
df.head(100)

event_key                                        event_value  \
0               deck  [FP1_027, EX1_509, NEW1_029, OG_134, GVG_071, ...   
1               deck  [GVG_081, AT_106, OG_090, AT_070, EX1_007, CS2...   
2               deck  [FP1_015, OG_133, EX1_572, EX1_558, NEW1_041, ...   
3               deck  [CS2_226, NEW1_019, CS2_186, BRM_033, FP1_030,...   
4               deck  [EX1_556, EX1_170, EX1_011, OG_254, EX1_044, G...   
5               deck  [EX1_009, OG_122, EX1_050, EX1_062, OG_321, GV...   
6               deck  [AT_101, EX1_341, EX1_284, EX1_085, LOE_111, G...   
7          deck_cost                                      3.95333333333   
8          deck_cost                                      3.95333333333   
9          deck_cost                                      3.95333333333   
10         deck_cost                                      3.95333333333   
11         deck_cost                                      3.95333333333   
12       num_minions                                                278   
13              deck  [AT_113, OG_174, CS1_069, GVG_095, EX1_045, AT...   
14         deck_cost                                      3.95333333333   
15         deck_cost                                      3.95333333333   
16       num_minions                                                278   
17       num_minions                                                278   
18       num_minions                                                278   
19        num_spells                                                 20   
20       num_minions                                                278   
21        num_spells                                                 20   
22         deck_cost                                      3.95333333333   
23       num_minions                                                278   
24        num_spells                                                 20   
25       num_weapons                                                  2   
26        num_spells                                                 20   
27        num_spells                                                 20   
28        num_spells                                                 20   
29       num_minions                                                278   
..               ...                                                ...   
70        num_spells                                                  1   
71       num_minions                                                  0   
72     player1_class                                   CardClass.PRIEST   
73        num_spells                                                  1   
74       num_weapons                                                  0   
75       num_weapons                                                  0   
76     player2_class                                    CardClass.DRUID   
77       num_weapons                                                  0   
78        num_spells                                                  0   
79      first_player                                            Player2   
80     player1_class                                  CardClass.WARLOCK   
81     player1_class                                  CardClass.PALADIN   
82     player1_class                                     CardClass.MAGE   
83       num_weapons                                                  0   
84     player2_class                                     CardClass.MAGE   
85     player2_class                                   CardClass.HUNTER   
86  cards_mulliganed                                   [Master Jouster]   
87       num_weapons                                                  1   
88      first_player                                            Player2   
89       num_weapons                                                  1   
90        cards_kept         [Shadowform, Lightspawn, Lost Tallstrider]   
91      first_player                

In [9]:
# creating a function to convert the format of the data from long to "longer"

def expand_decks(df):
    if len(df.event_value.values[0]) == 0:
        df['event_value'] = 'NONE'
        df['event_iter'] = 0.
        return df
    else:
        ndf = pd.concat([df]*len(df.event_value.values[0]), ignore_index=True)
        ndf['event_value'] = df.event_value.values[0]
        ndf['event_iter'] = range(ndf.shape[0])
        return ndf


In [10]:
# applying the above function to the event_values that are lists
decks_times = df.loc[df.event_key.isin(['deck','cards_mulliganed','cards_kept'])]
decks = decks_times.groupby('timestamp').apply(expand_decks).reset_index(drop=True)

In [15]:
decks.head(31)

event_key event_value game_id   player                   timestamp  \
0       deck     FP1_027       1  player1  2016-06-08 18:47:53:334438   
1       deck     EX1_509       1  player1  2016-06-08 18:47:53:334438   
2       deck    NEW1_029       1  player1  2016-06-08 18:47:53:334438   
3       deck      OG_134       1  player1  2016-06-08 18:47:53:334438   
4       deck     GVG_071       1  player1  2016-06-08 18:47:53:334438   
5       deck     EX1_583       1  player1  2016-06-08 18:47:53:334438   
6       deck     EX1_620       1  player1  2016-06-08 18:47:53:334438   
7       deck     EX1_009       1  player1  2016-06-08 18:47:53:334438   
8       deck     GVG_108       1  player1  2016-06-08 18:47:53:334438   
9       deck      OG_300       1  player1  2016-06-08 18:47:53:334438   
10      deck      AT_096       1  player1  2016-06-08 18:47:53:334438   
11      deck     CS2_181       1  player1  2016-06-08 18:47:53:334438   
12      deck    NEW1_025       1  player1  2016-06-08 18:47:53:334438   
13      deck     GVG_084       1  player1  2016-06-08 18:47:53:334438   
14      deck      OG_286       1  player1  2016-06-08 18:47:53:334438   
15      deck     CS2_092       1  player1  2016-06-08 18:47:53:334438   
16      deck     GVG_117       1  player1  2016-06-08 18:47:53:334438   
17      deck    NEW1_018       1  player1  2016-06-08 18:47:53:334438   
18      deck      OG_284       1  player1  2016-06-08 18:47:53:334438   
19      deck     CS1_042       1  player1  2016-06-08 18:47:53:334438   
20      deck      AT_127       1  player1  2016-06-08 18:47:53:334438   
21      deck     EX1_029       1  player1  2016-06-08 18:47:53:334438   
22      deck     GVG_064       1  player1  2016-06-08 18:47:53:334438   
23      deck     EX1_363       1  player1  2016-06-08 18:47:53:334438   
24      deck     EX1_284       1  player1  2016-06-08 18:47:53:334438   
25      deck     EX1_097       1  player1  2016-06-08 18:47:53:334438   
26      deck      OG_152       1  player1  2016-06-08 18:47:53:334438   
27      deck     EX1_382       1  player1  2016-06-08 18:47:53:334438   
28      deck      AT_113       1  player1  2016-06-08 18:47:53:334438   
29      deck     LOE_038       1  player1  2016-06-08 18:47:53:334438   
30      deck     GVG_081       1  player1  2016-06-08 18:47:53:370128   

    event_iter  
0          0.0  
1          1.0  
2          2.0  
3          3.0  
4          4.0  
5          5.0  
6          6.0  
7          7.0  
8          8.0  
9          9.0  
10        10.0  
11        11.0  
12        12.0  
13        13.0  
14        14.0  
15        15.0  
16        16.0  
17        17.0  
18        18.0  
19        19.0  
20        20.0  
21        21.0  
22        22.0  
23        23.0  
24        24.0  
25        25.0  
26        26.0  
27        27.0  
28        28.0  
29        29.0  
30         0.0

In [11]:
nodecks = df[~df.event_key.isin(['deck','cards_mulliganed','cards_kept'])]
nodecks['event_iter'] = 0
expanded = pd.concat([decks, nodecks], ignore_index=True)
expanded.sort_values(['event_key','event_iter','player','timestamp'], inplace=True)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
expanded.head(10)

event_key    event_value game_id   player  \
81118  avg_hand_cost           3.75       1  Player1   
81129  avg_hand_cost            3.4       1  Player1   
81141  avg_hand_cost  4.33333333333       1  Player1   
81140  avg_hand_cost            4.0       1  Player1   
81177  avg_hand_cost            1.5       1  Player1   
81206  avg_hand_cost            3.4       1  Player1   
81207  avg_hand_cost            2.4       1  Player1   
81202  avg_hand_cost            4.0       1  Player1   
81226  avg_hand_cost            3.5       1  Player1   
81253  avg_hand_cost           4.25       1  Player1   

                        timestamp  event_iter  
81118  2016-06-08 18:47:55:080150         0.0  
81129  2016-06-08 18:47:55:241967         0.0  
81141  2016-06-08 18:47:55:405598         0.0  
81140  2016-06-08 18:47:55:455376         0.0  
81177  2016-06-08 18:47:55:795405         0.0  
81206  2016-06-08 18:47:56:128586         0.0  
81207  2016-06-08 18:47:56:165903         0.0  
81202  2016-06-08 18:47:56:180581         0.0  
81226  2016-06-08 18:47:56:453777         0.0  
81253  2016-06-08 18:47:56:664127         0.0